# CARA metabolomics - Establish putative microbial secondary metabolites - FBMN

**Authors**: Louis Felix Nothias, Feb 2021

**Objective**: 
- We match putative spectral annotation against NPAtlas and MIBIG and recover metadata (prefix `NPA_` and `MIBIG_` respectively)

- We create new columns to indicate if an annotation is microbial (`is_microbial`). We differenciate the annotation level based on Metabolomics Standard Initiative standards. We also create new columns that store the identifiers from NPAtlas and MiBIG.
    - Level 2: GNPS spectral library match in regular and analaogue mode, for `GNPS_LIB` and `GNPS_LIBA`
    - Level 4: SIRIUS/CSI:FingerID, for `CSI_`

   For example `is_microbial_level_2` or by combining levels `is_microbial_level_2_4`. 'YES' indicates this metabolites is potentially microbial and belong to the respective annotation level.
   
   `is_microbial_tool` column summarizes the annotation tool that gave a microbial metabolite hits.
   
   `is_microbial_tool_id` column summarizes the NPAtlas and MIBIG identifiers for the hits.

- We propagate the microbial molecules using molecular network families (using `GNPS_componentindex`) and create a new column to indicate that these molecules are part of a putative microbial network. For example with columns: `is_microbial_level_2_4_network` and `is_microbial_level_2_4_networkid` (for id of `GNPS_componentindex`).

- We create consensus columns for SMILES, annotation tool, name, chemical classs with prefix for columns `Cons_`

In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem

In [2]:
feature_metadata_table = 'Input/FBMN/HILICneg_feature_metadata_consolidated.tsv'
features = pd.read_csv(feature_metadata_table,
                       sep='\t', low_memory=False)

In [3]:
npatlas = pd.read_csv('microbial_metabolite_database/NPAtlas_download.tsv', sep='\t')
new_names = [(i,'NPA_'+i) for i in npatlas.iloc[:, 0:].columns.values]
npatlas.rename(columns = dict(new_names), inplace=True)

In [4]:
# This prints the metadata columns per annotations using the respective prefix.
def show_metadata_tools(table, metadata_prefix):
    metadata = []
    for x in table.columns:
        if x.startswith(metadata_prefix):
            metadata.append(str(x))
    print(metadata)

### Generate the inchikey first block for all the annotation tools

In [5]:
#Make a new column for inchikey without stereo
def _prepare_inchikey(table, column):
    table[column+str('_no_stereo')] = (table[column]
                                .str.split('-').str[0])

In [6]:
#Input columns for inchikey
gnps_lib = 'GNPS_LIB_Consol_InChIKey'
gnps_liba = 'GNPS_LIBA_Consol_InChIKey'
csi = 'CSI_InChIkey2D'

In [7]:
#Apply to the tables

    #NPAtlas
_prepare_inchikey(npatlas, 'NPA_compound_inchikey')

    #Feature annotation table
_prepare_inchikey(features, gnps_lib)
_prepare_inchikey(features, gnps_liba)
_prepare_inchikey(features, csi)

### Aggregate and match feature metadata with NPAtlas

In [8]:
# Aggregate and merge. Beautiful piece by Wout Bittremieux
def _aggregate_npatlas(table, npatlas, column, prefix):
    npatlas2 = npatlas
    npatlas2 = (npatlas2.groupby('NPA_compound_inchikey_no_stereo')
               [['NPA_npaid', 'NPA_compound_id', 'NPA_compound_names', 'NPA_origin_type',
                 'NPA_genus', 'NPA_origin_species', 'NPA_mibig_ids']]
               .agg(lambda values: '|'.join([str(v) for v in values]))
               .reset_index())
    new_names = [(i,str(prefix)+i) for i in npatlas2.iloc[:, 0:].columns.values]
    npatlas2.rename(columns = dict(new_names), inplace=True)
    _aggregate_npatlas.merged = pd.merge(table, npatlas2, left_on=column+'_no_stereo', right_on=str(prefix)+'NPA_compound_inchikey_no_stereo', how ='left')

In [9]:
# Apply to each annotation tool
_aggregate_npatlas(features, npatlas, gnps_lib,'GNPS_LIB_')
_aggregate_npatlas(_aggregate_npatlas.merged, npatlas, gnps_liba,'GNPS_LIBA_')
_aggregate_npatlas(_aggregate_npatlas.merged, npatlas, csi,'CSI_')

## Aggregate and match with MIBIG

In [10]:
mibig = pd.read_csv('microbial_metabolite_database/mibig.csv', sep=',')
new_names = [(i,'MIBIG_'+i) for i in mibig.iloc[:, 0:].columns.values]
mibig.rename(columns = dict(new_names), inplace=True)

In [11]:
# Function to aggregate with MiBIG
def _aggregate_mibig(table, mibig, column, prefix):
    mibig2 = mibig
    mibig2 = (mibig2.groupby('MIBIG_no_stereo_inchikey')
               [['MIBIG_mibig_accession', 'MIBIG_organism_name','MIBIG_compound_name','MIBIG_ncbi_tax_id']]
               .agg(lambda values: '|'.join([str(v) for v in values]))
               .reset_index())
    new_names = [(i,str(prefix)+i) for i in mibig2.iloc[:, 0:].columns.values]
    mibig2.rename(columns = dict(new_names), inplace=True)
    _aggregate_mibig.merged = pd.merge(table, mibig2, left_on=column+'_no_stereo', right_on=str(prefix)+'MIBIG_no_stereo_inchikey', how ='left')

In [12]:
# Apply to each annotation tool
_aggregate_mibig(_aggregate_npatlas.merged, mibig, gnps_lib,'GNPS_LIB_')
_aggregate_mibig(_aggregate_mibig.merged, mibig, gnps_liba,'GNPS_LIBA_')
_aggregate_mibig(_aggregate_mibig.merged, mibig, csi,'CSI_')

## We check at what MSI level the microbial annotation is

In [13]:
table = _aggregate_mibig.merged

In [14]:
level_2 = []
level_4 = []
level_2_4 = []

# We are making columns to indicate if the compound is microbial and the annotation level of the tool
for i, row in table.iterrows():
    #level_2
    if row['GNPS_LIB_NPA_compound_id'] is not np.nan:
        level_2.append('YES')
    elif row['GNPS_LIBA_NPA_compound_id'] is not np.nan:
        level_2.append('YES')
    elif row['GNPS_LIB_MIBIG_mibig_accession'] is not np.nan:
        level_2.append('YES')
    elif row['GNPS_LIBA_MIBIG_mibig_accession'] is not np.nan:
        level_2.append('YES')
    else:
        level_2.append(np.nan)
        
    #level_4     
    if row['CSI_NPA_compound_id'] is not np.nan:
        level_4.append('YES')
    elif row['CSI_MIBIG_mibig_accession'] is not np.nan:
        level_4.append('YES')
    else:
        level_4.append(np.nan)
    
    #level_2_4
    if row['GNPS_LIB_NPA_compound_id'] is not np.nan:
        level_2_4.append('YES')
    elif row['GNPS_LIBA_NPA_compound_id'] is not np.nan:
        level_2_4.append('YES')
    elif row['CSI_NPA_compound_id'] is not np.nan:
        level_2_4.append('YES')
    elif row['GNPS_LIB_MIBIG_mibig_accession'] is not np.nan:
        level_2_4.append('YES')
    elif row['GNPS_LIBA_MIBIG_mibig_accession'] is not np.nan:
        level_2_4.append('YES')
    elif row['CSI_MIBIG_mibig_accession'] is not np.nan:
        level_2_4.append('YES')
    else:
        level_2_4.append(np.nan)
        
table['is_microbial_level_2'] = level_2
table['is_microbial_level_4'] = level_4
table['is_microbial_level_2_4'] = level_2_4

In [15]:
table.head(5)

,Unnamed: 0,#featureID,GNPS_Annotated Adduct Features ID,GNPS_Best Ion,GNPS_Correlated Features Group ID,GNPS_G1,GNPS_G2,GNPS_G3,GNPS_G4,GNPS_G5,...,GNPS_LIBA_MIBIG_compound_name,GNPS_LIBA_MIBIG_ncbi_tax_id,CSI_MIBIG_no_stereo_inchikey,CSI_MIBIG_mibig_accession,CSI_MIBIG_organism_name,CSI_MIBIG_compound_name,CSI_MIBIG_ncbi_tax_id,is_microbial_level_2,is_microbial_level_4,is_microbial_level_2_4
0,0,1,,,,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,,,19.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,,,,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,,,,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,,,,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
print('level_2 = '+ str(table[(table['is_microbial_level_2'] == 'YES')].shape[0]))
print('level_4 = '+ str(table[(table['is_microbial_level_4'] == 'YES')].shape[0]))
print('level_2_4 = '+ str(table[(table['is_microbial_level_2_4'] == 'YES')].shape[0]))

level_2 = 39
level_4 = 9
level_2_4 = 48


### For microbial annotation, we make a column that store information about annotation and accession number


In [17]:
element_list = []
id_list = []

for i, row in table.iterrows():
    element = []
    identifier = []
    
    #Initiating 
    if row['GNPS_LIB_NPA_npaid'] is not np.nan:
        element = 'GNPS_LIB_NPA|'
        identifier = row['GNPS_LIB_NPA_npaid']+str('|')
    else:
        element = ''
        identifier = ''
    
    #NPAtlas
    if row['GNPS_LIBA_NPA_npaid'] is not np.nan:
        element += 'GNPS_LIBA_NPA|'
        identifier += row['GNPS_LIBA_NPA_npaid']+str('|')
    if row['CSI_NPA_npaid'] is not np.nan:
        element += 'CSI_NPA|'
        identifier += row['CSI_NPA_npaid']+str('|')
    
    #MIBIG
    if row['GNPS_LIB_MIBIG_mibig_accession'] is not np.nan:
        element += 'GNPS_LIB_MIBIG|'
        identifier += row['GNPS_LIB_MIBIG_mibig_accession']+str('|')
    if row['GNPS_LIBA_MIBIG_mibig_accession'] is not np.nan:
        element += 'GNPS_LIBA_MIBIG|'
        identifier += row['GNPS_LIBA_MIBIG_mibig_accession']+str('|')
    if row['CSI_MIBIG_mibig_accession'] is not np.nan:
        element += 'CSI_MIBIG|'
        identifier += row['CSI_MIBIG_mibig_accession']+str('|')
    
    element_list.append(element)
    id_list.append(identifier)
    
table['is_microbial_tool'] = element_list 
table['is_microbial_tool_id'] = id_list

### We are propagating microbial annotations using molecular networking

In [18]:
# We are propagating microbial annotation
def _propagate_microbial_annotation(table,column):

    propagate_list = []
    propagate_column = []
    propagate_componentid = []

    for i, row in table.iterrows():
        if row[column] is not np.nan:
            if row['GNPS_componentindex'] is not -1:
                propagate_list.append(row['GNPS_componentindex'])

    for i, row in table.iterrows():
        if row['GNPS_componentindex'] in propagate_list:
            propagate_column.append('YES')
            propagate_componentid.append(row['GNPS_componentindex'])
        else:
            propagate_column.append(np.nan)
            propagate_componentid.append(np.nan)
            
    table[column+'_network'] = propagate_column
    table[column+'_networkid'] = propagate_componentid

In [19]:
# We propagate for each annotation level
_propagate_microbial_annotation(table,'is_microbial_level_2')
_propagate_microbial_annotation(table,'is_microbial_level_4')
_propagate_microbial_annotation(table,'is_microbial_level_2_4')

In [20]:
print('level_2_network = '+ str(table[(table['is_microbial_level_2_network'] == 'YES')].shape[0]))
print('level_4_network = '+ str(table[(table['is_microbial_level_4_network'] == 'YES')].shape[0]))
print('level_2_4_network = '+ str(table[(table['is_microbial_level_2_4_network'] == 'YES')].shape[0]))

level_2_network = 373
level_4_network = 83
level_2_4_network = 408


## Preparing Consensus Columns for SMILES / name / class

In [21]:
Cons_SMILES = []
Cons_Name = []
Cons_Name_ClassyFire = []
Cons_Highest_annot = []

for i, row in table.iterrows():
    # We are making columns for Consensus SMILES
    if row['GNPS_LIB_Consol_SMILES'] is not np.nan:
        Cons_SMILES.append(row['GNPS_LIB_Consol_SMILES'])
    elif row['GNPS_LIBA_Consol_SMILES'] is not np.nan:
        Cons_SMILES.append(row['GNPS_LIBA_Consol_SMILES'])
    elif row['CSI_smiles'] is not np.nan:
        Cons_SMILES.append(row['CSI_smiles'])
    else:
        Cons_SMILES.append(np.nan)
    
    #Highest annotation and consensus compound name
    if row['GNPS_LIB_SpectrumID'] is not np.nan:
        Cons_Highest_annot.append('GNPS_LIB')
        Cons_Name.append(str(row['GNPS_precursor mass'])+' '+str(row['SIR_MF_Zod_molecularFormula'])+' '+row['GNPS_LIB_Compound_Name']) 
    elif row['GNPS_LIBA_SpectrumID'] is not np.nan:
        Cons_Highest_annot.append('GNPS_LIBA')
        Cons_Name.append(str(row['GNPS_precursor mass'])+' '+str(row['SIR_MF_Zod_molecularFormula'])+' '+row['GNPS_LIBA_Compound_Name'])
    elif row['CSI_smiles'] is not np.nan:
        Cons_Highest_annot.append('SIRIUS_CSI')
        Cons_Name.append(str(row['GNPS_precursor mass'])+' '+str(row['SIR_MF_Zod_molecularFormula'])+' '+str(row['CSI_name']))
    elif row['CAN_most specific class'] is not np.nan:
        Cons_Highest_annot.append('SIRIUS_CANOPUS')
        Cons_Name.append(str(row['GNPS_precursor mass'])+' '+str(row['SIR_MF_Zod_molecularFormula'])+' '+row['CAN_most specific class'])
    elif row['SIR_MF_Zod_molecularFormula'] is not np.nan:
        Cons_Highest_annot.append('SIRIUS_MF')
        Cons_Name.append(str(row['GNPS_precursor mass'])+' '+str(row['SIR_MF_Zod_molecularFormula']))
    elif row['GNPS_componentindex'] != -1:
        Cons_Highest_annot.append('NETWORK')
        Cons_Name.append(str(row['GNPS_precursor mass']))
    else:
        Cons_Highest_annot.append(np.nan)
        Cons_Name.append(str(row['GNPS_precursor mass']))
        
    #Highest class annotation compound name
    if row['GNPS_LIB_SpectrumID'] is not np.nan:
        Cons_Name_ClassyFire.append(str(row['GNPS_precursor mass'])+' '+str(row['GNPS_LIB_class'])+' '+str(row['GNPS_LIB_Adduct']))
    elif row['GNPS_LIBA_SpectrumID'] is not np.nan:
        Cons_Name_ClassyFire.append(str(row['GNPS_precursor mass'])+' '+str(row['GNPS_LIBA_class'])+' '+str(row['GNPS_LIBA_Adduct']))
    elif row['CAN_most specific class'] is not np.nan:
        Cons_Name_ClassyFire.append(str(row['GNPS_precursor mass'])+' '+str(row['CAN_most specific class'])+' '+str(row['SIR_MF_Zod_adduct']))
    elif row['SIR_MF_Zod_molecularFormula'] is not np.nan:
        Cons_Name_ClassyFire.append(str(row['GNPS_precursor mass'])+' '+str(row['SIR_MF_Zod_molecularFormula'])+' '+str(row['SIR_MF_Zod_adduct']))
    else:
        Cons_Name_ClassyFire.append(str(row['GNPS_precursor mass']))
                                                         
table['Cons_SMILES'] = Cons_SMILES
table['Cons_Highest_annot'] = Cons_Highest_annot
table['Cons_Name'] = Cons_Name
table['Cons_Name_ClassyFire'] = Cons_Name_ClassyFire
table['Cons_Name'] = table['Cons_Name'].str.replace(' nan', '')
table['Cons_Name_ClassyFire']= table['Cons_Name_ClassyFire'].str.replace(' nan', ' ')

In [24]:
table.head(3)

,Unnamed: 0,#featureID,GNPS_Annotated Adduct Features ID,GNPS_Best Ion,GNPS_Correlated Features Group ID,GNPS_G1,GNPS_G2,GNPS_G3,GNPS_G4,GNPS_G5,...,is_microbial_level_2_network,is_microbial_level_2_networkid,is_microbial_level_4_network,is_microbial_level_4_networkid,is_microbial_level_2_4_network,is_microbial_level_2_4_networkid,Cons_SMILES,Cons_Highest_annot,Cons_Name,Cons_Name_ClassyFire
0,0,1,,,,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIRIUS_MF,112.9855 C2HF3O2,112.9855 C2HF3O2 [M - H]-
1,1,2,,,19.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIRIUS_MF,359.1573 C14H24N4O7,359.1573 C14H24N4O7 [M - H]-
2,2,3,,,,0.0,0.0,0.0,0.0,0.0,...,YES,34.0,NaN,NaN,YES,34.0,C#CCCCCCCCCCCCC(O)CC(O)COC(C)=O,GNPS_LIB,311.168 C17H28O3S 5S-HETE-d8,311.168 M-H


### We write out the generated table

In [23]:
table_out = table.drop('Unnamed: 0', axis=1)
table_out.to_csv(feature_metadata_table[:-4]+'_is_microbial.tsv', sep='\t', index=False)